<a href="https://colab.research.google.com/github/popcorn-wataroger/study_langchain/blob/main/Q1_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
必要なライブラリのインストール

必要なモジュールのインストール

In [10]:
!pip install openai
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install langchain-chroma
!pip install chromadb
!pip install pypdf
!pip install unstructured python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.4/189.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.6/114.6 kB 10.2 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=f21c8578fa029635055b72d2fd5f3d44cfbf7a8e55d78f1d515e9632cd3aabfa
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


LLMシステム

In [22]:
from openai import OpenAI
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredWordDocumentLoader
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

import os
from google.colab import userdata
import numpy as np

OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# 類似度計算関数
def cosine_similarity(vec1, vec2):
    vec1, vec2 = np.array(vec1), np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 類似質問をベクトルで探索
def resolve_similar_question(user_input: str, question_history: list, embeddings_model, threshold: float = 0.85):
    if not question_history:
        return None

    query_vec = embeddings_model.embed_query(user_input)
    max_sim = -1
    best_question = None

    for q in question_history:
        q_vec = embeddings_model.embed_query(q)
        sim = cosine_similarity(query_vec, q_vec)
        if sim > max_sim:
            max_sim = sim
            best_question = q

    if max_sim >= threshold:
        print(f"\n→ 類似度 {max_sim:.2f} により過去の質問を再利用します：{best_question}")
        return best_question
    return None

# ドキュメント読み込み関数
def load_all_documents_from_folder(folder_path: str):
  documents = []
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    try:
      if filename.endswith(".pdf"):
          loader = PyPDFLoader(file_path)
      elif filename.endswith(".txt"):
          loader = TextLoader(file_path, encoding="utf-8")
      elif filename.endswith(".docx"):
          loader = UnstructuredWordDocumentLoader(file_path)
      else:
          print(f"サポート外ファイルのためスキップ: {filename}")
          continue

      docs = loader.load()
      documents.extend(docs)
      print(f"読み込み成功: {filename}（{len(docs)}件）")

    except Exception as e:
      print(f"読み込み失敗: {filename} - エラー内容: {str(e)}")

  return documents

# ドキュメント読み込み
documents = load_all_documents_from_folder("./data")

# ベクトルDB・LLM設定
embeddings_model = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model="text-embedding-3-small")
db = Chroma.from_documents(documents=documents, embedding=embeddings_model)
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-4o-mini")

# プロンプトテンプレート（内容をmessages履歴に組み込むため使わない）
system_prompt = """
あなたは複数のドキュメントに基づいて質問に答えるアシスタントです。
必要に応じてドキュメント内容を参照しながら、ユーザーの質問に丁寧に答えてください。
"""

# 会話履歴の保持
messages = [{"role": "system", "content": system_prompt}]
question_history, answer_history = [], []

while True:
  user_input = input("\n質問を入力してください（終了するには 'exit'）：\n> ")

  if user_input.lower() in ["exit"]:
    print("終了します。")
    break

  if not user_input:
    continue

  # 類似質問を探す（過去質問で類似度が高いもの）
  resolved_question = resolve_similar_question(user_input, question_history, embeddings_model)
  if resolved_question:
      user_input = resolved_question

  # 類似ドキュメント取得
  question_embedding = embeddings_model.embed_query(user_input)
  similar_docs = db.similarity_search_by_vector(question_embedding, k=3)
  docs_text = "\n\n".join([doc.page_content for doc in similar_docs])

  # プロンプト構成
  user_message = f"{user_input}以下は参考ドキュメントの抜粋です：\n{docs_text}"
  messages.append({"role": "user", "content": user_message})

  # LLM呼び出し
  response = llm.invoke(messages)
  assistant_reply = response.content
  messages.append({"role": "assistant", "content": assistant_reply})

  # 会話保持
  question_history.append(user_input)
  answer_history.append(assistant_reply)

  # 出力
  print("\n回答:")
  print(assistant_reply)

読み込み成功: 一億総活躍社会の実現に向けて.pdf（24件）
読み込み成功: 従業員からの介護相談の対応について.docx（1件）
サポート外ファイルのためスキップ: .ipynb_checkpoints
読み込み成功: 雇用・失業調整の動向.pdf（4件）

質問を入力してください（終了するには 'exit'）：
> 2020 年平均では有効求人倍率は前年差どうなってる？

回答:
2020年の平均有効求人倍率は、前年に比べて0.42ポイント低下し、1.18倍となりました。この影響は、COVID-19のパンデミックによる緊急事態宣言や行動制限が原因とされています。また、完全失業率は0.4ポイント上昇して2.8%になっています。

したがって、2020年は有効求人倍率が前年から減少しており、労働市場における厳しい状況が続いていたことが示されています。

質問を入力してください（終了するには 'exit'）：
> 雇用情勢は、2021 年以降、どうなってる？

回答:
2021年以降の雇用情勢は、徐々に持ち直している状況です。経済社会活動が回復しつつある中で、求人数の回復には一部の産業で遅れが見られるものの、全体として求人は増加傾向にあります。特に、女性や高齢者などの労働参加が進展していることも挙げられます。

2022年には、新規求人数が前年から2年連続で増加し、新規求人倍率は前年差0.24ポイント上昇の2.26倍、有効求人倍率は同0.15ポイント上昇の1.28倍となりました。これに加えて、完全失業率も0.2ポイント低下し2.6%となっていますが、いずれも感染拡大前の2019年の水準には回復していない状況です。

ただし、少子高齢化に伴う労働供給の制約や、人手不足の問題が再び顕在化している点に留意する必要があります。全体的には、経済活動の回復に合わせて雇用情勢も改善しているものの、課題は残されているという状況です。

質問を入力してください（終了するには 'exit'）：
> 従業員から介護に関する相談を受けた際に対応すべきことは？

回答:
従業員から介護に関する相談を受けた際に対応すべきポイントは、以下の6つにまとめられます。

1. **企業の姿勢を伝える**: 従業員の仕事と介護の両立を支援する企業姿勢をしっかりと伝え、サポートの意思を示します。

2. 